<img src='slides/4.png'>

In [1]:
import pandas as pd
import numpy as np

# ファイルの内容を確認
# - %pycat が使用できない場合は %pfile, %cat 等を試してください
%pycat data/14_1_cleansing.csv

引用符;カンマ付;非数値;16進数
'123';123,456;18;ABADBABE
'456';456,789;18歳未満;BAADF00D
'789';1,000,000;20;BADDCAFE
'012';2,000,000;30;CAFEBABE
'345';3,000,000;40;DEADBEEF


In [2]:
# DataFrameに読み込み
df = pd.read_csv('data/14_1_cleansing.csv', sep=';')

# 確認
df

,引用符,カンマ付,非数値,16進数
0,'123',"123,456",18,ABADBABE
1,'456',"456,789",18歳未満,BAADF00D
2,'789',"1,000,000",20,BADDCAFE
3,'012',"2,000,000",30,CAFEBABE
4,'345',"3,000,000",40,DEADBEEF


In [3]:
# 各列のデータ型を確認
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   引用符     5 non-null      object
 1   カンマ付    5 non-null      object
 2   非数値     5 non-null      object
 3   16進数    5 non-null      object
dtypes: object(4)
memory usage: 288.0+ bytes


In [4]:
# 'カンマ付'列の1行目
df['カンマ付'].iloc[0]

'123,456'

In [5]:
# 非数値（文字列）のため、数値との計算の対象にはならず、エラーになる
df['カンマ付'].iloc[0] + 1

TypeError: can only concatenate str (not "int") to str

In [6]:
# 'カンマ付'の型の確認
type(df['カンマ付'].iloc[0])

str

引用符、カンマ付、16進数の各列は、それぞれに応じた処理を行う

In [7]:
# 引用符列は左右の端にある ' を除去した後に、int型に変換
df['引用符'] = df['引用符'].str.strip("'").astype(int)

# カンマ付列は , を除去した後に、int型に変換
df['カンマ付'] = df['カンマ付'].str.replace(',', '').astype(int)

# 16進数列は16進数を指定して型変換
df['16進数'] = df['16進数'].apply(lambda x: int(x, 16))

# 確認
df

,引用符,カンマ付,非数値,16進数
0,123,123456,18,2880289470
1,456,456789,18歳未満,3131961357
2,789,1000000,20,3135097598
3,12,2000000,30,3405691582
4,345,3000000,40,3735928559


In [8]:
# 各列のデータ型を確認
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   引用符     5 non-null      int64 
 1   カンマ付    5 non-null      int64 
 2   非数値     5 non-null      object
 3   16進数    5 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 288.0+ bytes


引用符、カンマ付、16進数の列は、数値 int64 型に変換

非数値列は、数値と非数値が混ざっているため、数値のみと非数値のみに分離し、それぞれの列を作成する

- どう扱うかは、その後の処理依存

In [9]:
# 非数値列の数値化できないデータは NaN に変換した数値のみ列を追加
# - NaNはint型で表現できないため、列はfloat型になってしまう
df['数値のみ'] = pd.to_numeric(df['非数値'], errors='coerce')

# 数値のみ列で NaN の行について、非数値列の値を取り出して、非数値のみ列に追加
df['非数値のみ'] = df[np.isnan(df['数値のみ'])].loc[:, '非数値']

# 確認
df

,引用符,カンマ付,非数値,16進数,数値のみ,非数値のみ
0,123,123456,18,2880289470,18.0,NaN
1,456,456789,18歳未満,3131961357,NaN,18歳未満
2,789,1000000,20,3135097598,20.0,NaN
3,12,2000000,30,3405691582,30.0,NaN
4,345,3000000,40,3735928559,40.0,NaN


In [10]:
# 各列のデータ型を確認
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   引用符     5 non-null      int64  
 1   カンマ付    5 non-null      int64  
 2   非数値     5 non-null      object 
 3   16進数    5 non-null      int64  
 4   数値のみ    4 non-null      float64
 5   非数値のみ   1 non-null      object 
dtypes: float64(1), int64(3), object(2)
memory usage: 368.0+ bytes


In [11]:
# 非数値列が必要無いならば削除
df = df.drop('非数値', axis=1)

# 確認
df

,引用符,カンマ付,16進数,数値のみ,非数値のみ
0,123,123456,2880289470,18.0,NaN
1,456,456789,3131961357,NaN,18歳未満
2,789,1000000,3135097598,20.0,NaN
3,12,2000000,3405691582,30.0,NaN
4,345,3000000,3735928559,40.0,NaN
